In [1]:
from layout import Layout
from pulp import *
import numpy as np
import random
from copy import deepcopy
import plotly.graph_objects as go
import networkx as nx
import pickle
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [2]:
def render_warehouse(warehouse: Layout, assignment, product_frequency):
    storage_assignment = deepcopy(warehouse.layout_grid)
    walkable_locs = np.where(warehouse.layout_grid == 0)

    for x, y, z in zip(walkable_locs[0], walkable_locs[1], walkable_locs[2]):
        storage_assignment[x,y,z] = -100

    for i, loc in enumerate(assignment):
        storage_assignment[warehouse.nodes_list[loc]] = i 

    storage_frequence = deepcopy(warehouse.layout_grid)
    for i, loc in enumerate(assignment):
        storage_frequence[warehouse.nodes_list[loc]] = product_frequency[i]
    
    node_xyz = np.array([warehouse.pos_dict[v] for v in sorted(warehouse.graph)])
    edge_xyz = np.array([(warehouse.pos_dict[u], warehouse.pos_dict[v]) for u, v in warehouse.graph.edges()])

    data = []
    for x, y, z in zip(node_xyz.T[0], node_xyz.T[1], node_xyz.T[2]):
        data_point = storage_frequence[x,y,z]
        data.append(data_point)

    assignment_plt = []
    for x, y, z in zip(node_xyz.T[0], node_xyz.T[1], node_xyz.T[2]):
        assignment_point = storage_assignment[x,y,z]
        assignment_plt.append(assignment_point)

    # data_names = []
    # # print(assignment_plt)
    # for i in assignment_plt:
    #     # print(i)
    #     if i == -100.0 or i == -1:
    #         data_names.append(i)
    #     else:
    #         data_names.append(product_names[int(i)])


    trace = go.Scatter3d(customdata=np.stack((data,assignment_plt),axis=-1), hovertemplate = 'x: %{x}<br>y: %{y}<br>z: %{z}<br>Order Frequency: %{customdata[0]}<br>Product: %{customdata[1]}}',
        x = node_xyz.T[0], y = node_xyz.T[1], z = node_xyz.T[2],mode = 'markers', marker = dict(symbol="square",
            size = 12,
            color = data, # set color to an array/list of desired values
            colorscale = 'thermal')
        )
    layout = go.Layout(title = 'Warehouse Visualization')
    fig = go.Figure(data = [trace], layout = layout)
    fig.update_traces(marker_size = 5)
    return fig

In [3]:
warehouse = Layout(5, 3, 1, False, False, None) 
double_deep = "False"
path = os.path.join("data/cache/dist_mats/", "dist_mat_" + str(warehouse.layout_grid.shape) + "_" + str(double_deep) + ".npy")
if os.path.exists(path):
    dist_mat = np.load(path)
else:
    dist_mat = warehouse.gen_dist_mat(path)

In [4]:
warehouse.layout_grid[:,:,0]

array([[ 0.,  0.,  0.],
       [-1.,  0., -1.],
       [-1.,  0., -1.],
       [-1.,  0., -1.],
       [ 0.,  0.,  0.]])

In [5]:
size = len(warehouse.storage_locs)
product_pairs_frequency = np.zeros((size, size))
upper_bound = 9
for i in range(size):
    for j in range(size):
        if i == j: 
            product_pairs_frequency[i][j] = 0
        # since problem is a one-one problem, mapping (1,2) == (2,1) so there's no need to create duplicate mappings
        #if (j,i) in product_pairs_frequency: continue
        else:
            product_pairs_frequency[i][j] = random.randrange(0, upper_bound)

In [6]:
product_frequency = {}
for i in range(size):
    product_frequency[i] = product_pairs_frequency[i].sum()

In [7]:
n_storage_locs = len(warehouse.storage_locs)
#n_products = len(warehouse.storage_locs)
n_products = len(product_frequency.keys())

storage_locs = [warehouse.nodes_list.index(loc) for loc in warehouse.storage_locs]
products = [p for p in range(n_products)]
storage_loc_mapping = {i:loc for i,loc in enumerate(storage_locs)}
storage_locs_new = [i for i in storage_loc_mapping.keys()]
depot = 0

In [8]:
storage_locs

[6, 5, 9, 8, 12, 11]

In [9]:
storage_locs_new

[0, 1, 2, 3, 4, 5]

In [10]:
assignment = storage_locs

In [11]:
fig = render_warehouse(warehouse, assignment, product_frequency)

In [12]:
fig.show()

In [13]:
dist_mat_storage = dist_mat[storage_locs][:, storage_locs]

In [14]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model("SLAP_PA")

# decision variables
y_i_j = {}
for i in products:
    for j in storage_locs: 
        y_i_j[i, j] = model.addVar(vtype=GRB.BINARY, name=f"Produkt{i}_an_LP_{j}") 

# objective function

model.setObjective(gp.quicksum(y_i_j[h, j] * y_i_j[i, k] * dist_mat[j, k] * product_pairs_frequency[h, i] 
                               for h in products for i in products for j in storage_locs for k in storage_locs),  GRB.MINIMIZE)
#+ gp.quicksum(y_i_j[i,j] * dist_mat[0, j] * product_frequency[i] for i in products for j in storage_locs
# constraints
for j in storage_locs: #storage_locs_new
    model.addConstr(gp.quicksum(y_i_j[i, j] for i in products) == 1, name=f"Lagerplatz_{i}_constraint")

for i in products:
    model.addConstr(gp.quicksum(y_i_j[i, j] for j in storage_locs) == 1, name=f"Produkt_{j}_constraint")

model.optimize()

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 12 rows, 36 columns and 72 nonzeros
Model fingerprint: 0x0680ee1c
Model has 450 quadratic objective terms
Variable types: 0 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [8e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 360.0000000
Presolve time: 0.01s
Presolved: 462 rows, 486 columns, 1422 nonzeros
Variable types: 0 continuous, 486 integer (486 binary)



Root relaxation: objective 0.000000e+00, 23 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   12  360.00000    0.00000   100%     -    0s
H    0     0                     353.0000000    0.00000   100%     -    0s
     0     0    0.00000    0   12  353.00000    0.00000   100%     -    0s
     0     0    0.00000    0   12  353.00000    0.00000   100%     -    0s
     0     0    0.00000    0   12  353.00000    0.00000   100%     -    0s
     0     2   24.00000    0   12  353.00000   24.00000  93.2%     -    0s
*   25     9               8     346.0000000  209.00000  39.6%  60.8    0s

Cutting planes:
  Zero half: 1
  RLT: 218

Explored 53 nodes (2332 simplex iterations) in 0.22 seconds (0.16 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 346 353 360 

Optimal solution found (tolerance

In [15]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model("SLAP_PA")

# decision variables
y_i_j = {}
for i in products:
    for j in storage_locs: #storage_locs_new
        y_i_j[i, j] = model.addVar(vtype=GRB.BINARY, name=f"Produkt{i}_an_LP_{j}") 

# objective function
# model.setObjective(gp.quicksum(y_i_j[h, j] * y_i_j[i, k] * dist_mat_storage[j, k] * product_pairs_frequency[h, i]
#                                for h in products for i in products for j in storage_locs_new for k in storage_locs_new), GRB.MINIMIZE)

model.setObjective(gp.quicksum(y_i_j[h, j] * y_i_j[i, k] * dist_mat[j, k] * product_pairs_frequency[h, i]
                               for h in products for i in products for j in storage_locs for k in storage_locs) + gp.quicksum(y_i_j[i,j] * dist_mat[0, j] * product_frequency[i] for i in products for j in storage_locs),  GRB.MINIMIZE)
#                      

# constraints
for j in storage_locs: #storage_locs_new
    model.addConstr(gp.quicksum(y_i_j[i, j] for i in products) == 1, name=f"Lagerplatz_{i}_constraint")

for i in products:
    model.addConstr(gp.quicksum(y_i_j[i, j] for j in storage_locs) == 1, name=f"Produkt_{j}_constraint") #storage_locs_new


In [16]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 12 rows, 36 columns and 72 nonzeros
Model fingerprint: 0xe09dc5fa
Model has 450 quadratic objective terms
Variable types: 0 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 1e+02]
  QObjective range [8e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 822.0000000
Presolve time: 0.00s
Presolved: 462 rows, 486 columns, 1422 nonzeros
Variable types: 0 continuous, 486 integer (486 binary)

Root relaxation: objective 4.550000e+02, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  455.000

In [17]:
model.objVal

810.0

In [18]:
# Print the optimal solution
if model.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    for i in products:
        for j in storage_locs:
            if y_i_j[i, j].x > 0.5:  # Check if the variable is assigned to 1 (approximately)
                print(f"Item {i} stored at storage location {j}")
else:
    print("No optimal solution found.") 

Optimal Solution Found:
Item 0 stored at storage location 9
Item 1 stored at storage location 8
Item 2 stored at storage location 12
Item 3 stored at storage location 11
Item 4 stored at storage location 6
Item 5 stored at storage location 5


In [19]:
product_pairs_frequency

array([[0., 4., 2., 4., 2., 6.],
       [8., 0., 1., 6., 4., 0.],
       [0., 5., 0., 5., 4., 2.],
       [6., 0., 5., 0., 2., 3.],
       [3., 6., 3., 6., 0., 2.],
       [5., 3., 8., 5., 8., 0.]])

In [20]:
# Close the Gurobi model
model.close()

In [21]:
import ga

In [22]:
def fitness_pf_pa(individual, dist_mat, product_affinity):
    affinity_score = 0
    for p in range(len(individual)):
        for p2 in range(len(individual)):
            if p == p2 or product_affinity[p][p2] == 0:
                continue
            else:
                affinity_score += product_affinity[p][p2] * dist_mat[individual[p]][individual[p2]]
    return affinity_score

In [23]:
def generate_genome(products, locations_list, layout: Layout):
    chromosome = []
   
    random.shuffle(locations_list)
    for i in range(len(products)):
        storage_loc = locations_list[i]
        chromosome.append(storage_loc)
    return chromosome

In [24]:
initial_population = [generate_genome(products, storage_locs, warehouse) for i in range(200)]

In [25]:
product_frequency

{0: 18.0, 1: 19.0, 2: 16.0, 3: 16.0, 4: 20.0, 5: 29.0}

In [26]:
# initial_population = [ga.generate_genome(products, warehouse) for i in range(1000)]

    
#product_pairs_frequency = calc_product_affinity(orders)
# product_frequency_pf_pa = [0 for i in range(len(product_frequency.keys()))]
# for i in list(product_frequency.values()):
product_frequency_pf_pa = list(product_frequency.values())



# product_pairs_np = np.zeros((n_products, n_products))
# for key in product_pairs_frequency.keys():
#     for pair in product_pairs_frequency[key].keys():
#         product_pairs_np[key][pair] = product_pairs_frequency[key][pair]

fitness_tracker = []
min_fitness = np.inf
for g in range(500):
    new_pop = []
    pop_results = []
    
   
    fitness_result = [ga.fitness_pf_pa(pop, dist_mat, product_frequency_pf_pa, product_pairs_frequency) for pop in initial_population]
    print(g, min(fitness_result))
    fitness_tracker.append(min(fitness_result))
    for _ in range(100):
        selected = ga.selection(initial_population, fitness_result)
        child = ga.crossover(selected)
        mutated = ga.mutation(child)
        new_pop.append(mutated)
    initial_population = new_pop

min_fitness = min(fitness_result)
min_child = initial_population[fitness_result.index(min_fitness)]

0 810.0
1 810.0
2 810.0
3 810.0
4 810.0
5 810.0
6 810.0
7 810.0
8 810.0
9 810.0
10 810.0
11 810.0
12 810.0
13 810.0
14 810.0
15 810.0
16 810.0
17 810.0
18 810.0
19 810.0
20 810.0
21 810.0
22 810.0
23 810.0
24 810.0
25 810.0
26 810.0
27 810.0
28 810.0
29 810.0
30 810.0
31 810.0
32 810.0
33 810.0
34 810.0
35 810.0
36 810.0
37 810.0
38 810.0
39 810.0
40 810.0
41 810.0
42 810.0
43 810.0
44 810.0
45 810.0
46 810.0
47 810.0
48 810.0
49 810.0
50 810.0
51 810.0
52 810.0
53 810.0
54 810.0
55 810.0
56 810.0
57 810.0
58 810.0
59 810.0
60 810.0
61 810.0
62 810.0
63 810.0
64 810.0
65 810.0
66 810.0
67 810.0
68 810.0
69 810.0
70 810.0
71 810.0
72 810.0
73 810.0
74 810.0
75 810.0
76 810.0
77 810.0
78 810.0
79 810.0
80 810.0
81 810.0
82 810.0
83 810.0
84 810.0
85 810.0
86 810.0
87 810.0
88 810.0
89 810.0
90 810.0
91 810.0
92 810.0
93 810.0
94 810.0
95 810.0
96 810.0
97 810.0
98 810.0
99 810.0
100 810.0
101 810.0
102 810.0
103 810.0
104 810.0
105 810.0
106 810.0
107 810.0
108 810.0
109 810.0
110 810.0


In [27]:
min_child

[8, 12, 11, 9, 5, 6]